In [125]:
import pandas as pd
import numpy as np
import sqlite3 as sql
import re
import os

# 1.Load sqlite database

In [45]:
def load_dataset(db, table):
    """
    db: str, sqlite3 databes file name
    table: str, sqlite table name
    """
    dataset = pd.read_sql(f"SELECT * FROM {table}", sql.connect(db))
    dataset['file_size'] = dataset['file_size'] / (1024 * 1024)
    dataset['file_size'] =  dataset['file_size'].astype(int)
    return dataset

In [47]:
dataset = load_dataset('dataset/database.db', 'movies_data')
dataset

,message_id,file_id,caption,file_size,id
0,2220,BAACAgEAAx0Ec8rVjQACCKxkZJmKt-leb7SDbZdArKJYCn...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,1
1,2221,BAACAgQAAx0Ec8rVjQACCK1kZJmL8H8uaPwcuC4W7tnJdL...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,2
2,2222,BAACAgEAAx0Ec8rVjQACCK5kZJmLSq-hTSSIxuVZyqhU--...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,3
3,2223,BAACAgIAAx0Ec8rVjQACCK9kZJmM3EGy87VlBnsuiyEk4L...,"#tarjima, #romantik #melodramma\n\nNomi: Shamo...",539,4
4,2224,BAACAgQAAx0Ec8rVjQACCLBkZJmNqK19bYLVLzBEIpKq-v...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,5
...,...,...,...,...,...
286,2506,BAACAgIAAx0Ec8rVjQACCcpkZJ5Yf-nkMzrWpF3Z3laa34...,Mister Alfons o'zbek film\n\n🔍 Kino topib beru...,279,287
287,2507,BAACAgIAAx0Ec8rVjQACCctkZJ5YSyz9Yq7Y7qE3t_C_lg...,Menga esa sen kerak \n\n🔍 Kino topib beruvchi ...,408,288
288,2508,BAACAgQAAx0Ec8rVjQACCcxkZJ5ZH1JeeR1qh4NDi-U3cL...,Shallaqi (o'zbek film) \n\n🔍 Kino topib beruvc...,378,289
289,2509,BAACAgQAAx0Ec8rVjQACCc1kZJ5ZgFmvGKi7oReLx--9cq...,3-kvartal (o'zbek film) \n\n🔍 Kino topib beruv...,240,290


# 2.Colect title

In [75]:
def clean_title(dataset_title):
    respond = []
    for caption in dataset_title.str.split('\n'):
        n = len(caption)
        if re.search('premyera', caption[0].lower()) and n >= 2:
            if caption[1].strip() == '' and n >= 3:
                respond.append(caption[2])
            else:
                respond.append(caption[1]) 
        else:
            if len(caption[0]) >= 50:
                respond.append(caption[0][0:49])
            else:
                respond.append(caption[0])
    return respond

In [76]:
dataset['title'] = clean_title(dataset.caption)
dataset

,message_id,file_id,caption,file_size,id,title
0,2220,BAACAgEAAx0Ec8rVjQACCKxkZJmKt-leb7SDbZdArKJYCn...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,1,Zorro
1,2221,BAACAgQAAx0Ec8rVjQACCK1kZJmL8H8uaPwcuC4W7tnJdL...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,2,Qonli tatil
2,2222,BAACAgEAAx0Ec8rVjQACCK5kZJmLSq-hTSSIxuVZyqhU--...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,3,Qo'rqmas
3,2223,BAACAgIAAx0Ec8rVjQACCK9kZJmM3EGy87VlBnsuiyEk4L...,#tarjima #romantik #melodramma\n\nNomi: Shamol...,539,4,#tarjima #romantik #melodramma
4,2224,BAACAgQAAx0Ec8rVjQACCLBkZJmNqK19bYLVLzBEIpKq-v...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,5,Nomi: Qonli O'yin yohud qimirlagan O'ladi
...,...,...,...,...,...,...
286,2506,BAACAgIAAx0Ec8rVjQACCcpkZJ5Yf-nkMzrWpF3Z3laa34...,Mister Alfons o'zbek film\n\n,279,287,Mister Alfons o'zbek film
287,2507,BAACAgIAAx0Ec8rVjQACCctkZJ5YSyz9Yq7Y7qE3t_C_lg...,Menga esa sen kerak \n\n,408,288,Menga esa sen kerak
288,2508,BAACAgQAAx0Ec8rVjQACCcxkZJ5ZH1JeeR1qh4NDi-U3cL...,Shallaqi (o'zbek film) \n\n,378,289,Shallaqi (o'zbek film)
289,2509,BAACAgQAAx0Ec8rVjQACCc1kZJ5ZgFmvGKi7oReLx--9cq...,3-kvartal (o'zbek film) \n\n,240,290,3-kvartal (o'zbek film)


In [77]:
stop_words = [
              r"19[0-9][0-9]|20[0-2][0-9]",
              r"o'zbek film|Uzbek tilida|Hind kino",
              r"O'zbek tilida|Uzbek kino|o'zbek kino|o'zbek кино",
              r"Nomi.*:|Kino nomi:|NOMI|Davlat :",
              r"🖥|🌎|🚑|🩺|🎞|📺|🎬|🎥|b/f|#|'|HD|»|«|▷|🤡|➺",
#               r"Ozbek Tilida|[SD]|【|】",
              r"[.*]|✎|Kino:|Film noмi:|\[\]|📹|✅|🔥|❓",
              r"[0-9][0-9][0-9]p|Multserial:|☃️|❄️|\(|\)",
              r"🔸|🔶|🍿|🔰|\[\]|✏️|Nomi|🇺🇿 📆 Yili 📁 Formati",
              r"Oʻzbek tilida|💎|.remyera|PREMYERA|𝐦𝐟|𝐦/𝐟 |m/s",
              r"➣|Ozbek.*$|🇺🇿|RetroSerial|RetroMultserial|Uzbek.*$",
              r":|B/F|,"]


In [78]:
def clear_title2(dataset_title, matches):
    for match in matches:
        dataset_title = dataset_title.str.replace(match, '', regex = True)
    return dataset_title

In [79]:
dataset['title'] = clear_title2(dataset.title, stop_words)
dataset

,message_id,file_id,caption,file_size,id,title
0,2220,BAACAgEAAx0Ec8rVjQACCKxkZJmKt-leb7SDbZdArKJYCn...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,1,Zorro
1,2221,BAACAgQAAx0Ec8rVjQACCK1kZJmL8H8uaPwcuC4W7tnJdL...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,2,Qonli tatil
2,2222,BAACAgEAAx0Ec8rVjQACCK5kZJmLSq-hTSSIxuVZyqhU--...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,3,Qorqmas
3,2223,BAACAgIAAx0Ec8rVjQACCK9kZJmM3EGy87VlBnsuiyEk4L...,#tarjima #romantik #melodramma\n\nNomi: Shamol...,539,4,tarjima romantik melodramma
4,2224,BAACAgQAAx0Ec8rVjQACCLBkZJmNqK19bYLVLzBEIpKq-v...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,5,Qonli Oyin yohud qimirlagan Oladi
...,...,...,...,...,...,...
286,2506,BAACAgIAAx0Ec8rVjQACCcpkZJ5Yf-nkMzrWpF3Z3laa34...,Mister Alfons o'zbek film\n\n,279,287,Mister Alfons
287,2507,BAACAgIAAx0Ec8rVjQACCctkZJ5YSyz9Yq7Y7qE3t_C_lg...,Menga esa sen kerak \n\n,408,288,Menga esa sen kerak
288,2508,BAACAgQAAx0Ec8rVjQACCcxkZJ5ZH1JeeR1qh4NDi-U3cL...,Shallaqi (o'zbek film) \n\n,378,289,Shallaqi
289,2509,BAACAgQAAx0Ec8rVjQACCc1kZJ5ZgFmvGKi7oReLx--9cq...,3-kvartal (o'zbek film) \n\n,240,290,3-kvartal


In [80]:
state = 0
if state == 1:
    for title in dataset['title']:
        print("------")
        print(title)
else:
    for title in clear_title2(dataset['title'], stop_words):
        print("------")
        print(title)

------
Zorro
------
Qonli tatil
------
Qorqmas
------
tarjima romantik melodramma
------
 Qonli Oyin yohud qimirlagan Oladi
------
Yulduzlar Aybdormi
------
Omad
------
Yovuz oʻliklar 2
------
Yovuz oʻliklar 2
------
Shoqol 
------
Shoqol 
------
Yovuz oʻliklar 2
------
Yovuz oʻliklar 2
------
Yovuz oʻliklar 2
------
Shoqol 
------
Shoqol 
------
 Chupa
------
 Chupa
------
 Chupa
------
Имя 
------
Morgdagi qotillik  
------
Morgdagi qotillik  
------
G‘alati narsalar | Ajabtovur ishlar 13-Qism
------
G‘alati narsalar | Ajabtovur ishlar 14-Qism
------
G‘alati narsalar | Ajabtovur ishlar 15-Qism
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Biz hammamiz olikmiz 🧟‍♂
------
Daxsha

# 3. Clean Caption

In [81]:
def clean_caption(dataset_caption, matches):
    for match in matches:
        dataset_caption = dataset_caption.str.replace(match, '', regex=True)
    return dataset_caption

In [82]:
matches = [
          r'@.* ',
          r'@.*\n',
          r'@Jahon_Primyeralar',
          r'@Tarjima_Kinolar_bbot',
          r"https://telegram.me/+(.*)([a-z])",
          r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]",
          r'@KINOLAR_VA_SERIALLAR_22',
          r'👉 i',
          r'Diniy savollar',
          r'🔍 Kino topib beruvchi bot:',
          r'Kanalga ulanish👇👇👇',
          r'🌐Kanalimiz: @TaronaMove',
          r'🔍Musiqa Bot: 🎧',
          r"🌐 Kanalimiz:",
          r'👉 👈',
          r'👇👉',
          r'👇|,'
          ]

In [83]:
dataset['caption'] = clean_caption(dataset.caption, matches)
for caption in dataset['caption']:
    print("-----")
    print(caption)

-----
Zorro

Tili: oʻzbek tilida
Janri: jangari sarguzasht
Sifati: 720p

Kino qidirish

📥
-----
Qonli tatil

Tili: oʻzbek tilida
Janri: komediya qoʻrqinchli
Sifati: 720p
Davlati: AQSh
Yili: 2022

Kino qidirish

📥
-----
Qo'rqmas

Tili: O'zbek tilida
Sifati  480p
Yili 2003
Davlat: AQSH
-----
#tarjima #romantik #melodramma

Nomi: Shamol Shiddati

Davlati: Janubiy Koreya
Sanasi: 2017
Tili: O'zbek tilida (Tarjima)

Kino qidirish

📥
-----
Nomi: Qonli O'yin yohud qimirlagan O'ladi

Davlati: Yaponiya
Sanasi: 2021
Tili: O'zbek tilida (Tarjima)

Kino qidirish

📥
-----
Yulduzlar Aybdormi

Davlati: Aqsh
Sanasi: 2016
Tili: O'zbek tilida (Tarjima)

Kino qidirish

📥
-----
Omad

Tili: oʻzbek tilida
Janri: komediya Sarguzasht multfilm
Sifati: 480p
Yili: 2022
Davlati: AQSh

Kino qidirish

📥
-----
Yovuz oʻliklar 2

Tili: oʻzbek tilida
Janri: triller Horror 
Sifati: 720p

Kino qidirish

Yangiliklar
👉 
👉 
-----
Yovuz oʻliklar 2

Tili: oʻzbek tilida
Janri: triller Horror 
Sifati: 1080p

Kino qidirish

Yangi

# 4.Seperate Series

In [66]:
def movi_or_series(dataset_caption):
    n = 0
    series = []
    movies = []

    for caption in dataset['caption']:
        if re.search(r"serial|G‘alati narsalar|Jumong|SHUNQOR|Quyosh avlodlari", caption):
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        elif re.search(r"Vorislar|Soʻl Yonim|Sirli.Olam|seriya|SHIFOKOR QISMATI", caption): #sayyoralik mahbubim 
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        elif re.search(r"sayyoralik mahbubim", caption):
            n+=1
            print("--------")
            print(caption)
            series.append(True)
            movies.append(False)
        else:
            series.append(False)
            movies.append(True)
    return series, movies, n

In [67]:
# series, movies, n = movi_or_series(dataset.caption)
# series_dataset = dataset[series]
# series_dataset.reset_index()

In [36]:
# dataset_movies.to_csv('dataset_movies.csv', index = False)
# series_dataset.to_csv('series_dataset.csv', index = False)

# 5. Set photos id

In [150]:
dataset.loc[:,'title'] = dataset['title'].astype(str).str.strip()

In [151]:
dataset

,message_id,file_id,caption,file_size,id,title,photos
0,2220,BAACAgEAAx0Ec8rVjQACCKxkZJmKt-leb7SDbZdArKJYCn...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,1,Zorro,2220.jpg
1,2221,BAACAgQAAx0Ec8rVjQACCK1kZJmL8H8uaPwcuC4W7tnJdL...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,2,Qonli tatil,2221.jpg
2,2222,BAACAgEAAx0Ec8rVjQACCK5kZJmLSq-hTSSIxuVZyqhU--...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,3,Qorqmas,2222.jpg
3,2223,BAACAgIAAx0Ec8rVjQACCK9kZJmM3EGy87VlBnsuiyEk4L...,#tarjima #romantik #melodramma\n\nNomi: Shamol...,539,4,tarjima romantik melodramma,2223.jpg
4,2224,BAACAgQAAx0Ec8rVjQACCLBkZJmNqK19bYLVLzBEIpKq-v...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,5,Qonli Oyin yohud qimirlagan Oladi,2224.jpg
...,...,...,...,...,...,...,...
286,2506,BAACAgIAAx0Ec8rVjQACCcpkZJ5Yf-nkMzrWpF3Z3laa34...,Mister Alfons o'zbek film\n\n,279,287,Mister Alfons,2506.jpg
287,2507,BAACAgIAAx0Ec8rVjQACCctkZJ5YSyz9Yq7Y7qE3t_C_lg...,Menga esa sen kerak \n\n,408,288,Menga esa sen kerak,2507.jpg
288,2508,BAACAgQAAx0Ec8rVjQACCcxkZJ5ZH1JeeR1qh4NDi-U3cL...,Shallaqi (o'zbek film) \n\n,378,289,Shallaqi,2508.jpg
289,2509,BAACAgQAAx0Ec8rVjQACCc1kZJ5ZgFmvGKi7oReLx--9cq...,3-kvartal (o'zbek film) \n\n,240,290,3-kvartal,2509.jpg


In [152]:
photos

['2302',
 '2464',
 '2470',
 '2316',
 '2458',
 '2276',
 '2510',
 '2504',
 '2262',
 '2289',
 '2288',
 '2505',
 '2263',
 '2277',
 '2459',
 '2471',
 '2317',
 '2303',
 '2465',
 '2315',
 '2473',
 '2467',
 '2301',
 '2329',
 '2498',
 '2261',
 '2507',
 '2275',
 '2249',
 '2248',
 '2274',
 '2260',
 '2506',
 '2499',
 '2328',
 '2466',
 '2300',
 '2314',
 '2472',
 '2338',
 '2476',
 '2310',
 '2304',
 '2462',
 '2489',
 '2258',
 '2502',
 '2264',
 '2270',
 '.DS_',
 '2271',
 '2503',
 '2265',
 '2259',
 '2488',
 '2305',
 '2463',
 '2477',
 '2311',
 '2339',
 '2449',
 '2461',
 '2307',
 '2313',
 '2475',
 '2273',
 '2267',
 '2501',
 '2298',
 '2299',
 '2266',
 '2500',
 '2272',
 '2312',
 '2474',
 '2460',
 '2306',
 '2448',
 '2375',
 '2349',
 '2229',
 '2228',
 '2348',
 '2374',
 '2410',
 '2376',
 '2438',
 '2389',
 '2388',
 '2439',
 '2363',
 '2377',
 '2429',
 '2373',
 '2415',
 '2367',
 '0404',
 '2399',
 '2366',
 '2372',
 '2414',
 '2428',
 '2358',
 '2364',
 '2416',
 '2370',
 '2238',
 '2239',
 '2417',
 '2371',
 '2365',
 

In [153]:
def creat_photos(dataset_message_id):
    photos = []
    folder_path = 'photos'
    for filename in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, filename)):
            id = filename[:4]
            photos.append(id)
    photos_id = []
    for message_id in dataset_message_id:
        if str(message_id) in photos:
            photos_id.append(f"{message_id}.jpg")
        else:
            photos_id.append("0404.jpg")
    return photos_id

In [154]:
dataset['photos'] = creat_photos(dataset.message_id.copy())
dataset

,message_id,file_id,caption,file_size,id,title,photos
0,2220,BAACAgEAAx0Ec8rVjQACCKxkZJmKt-leb7SDbZdArKJYCn...,Zorro\n\nTili: oʻzbek tilida\nJanri: jangari s...,819,1,Zorro,2220.jpg
1,2221,BAACAgQAAx0Ec8rVjQACCK1kZJmL8H8uaPwcuC4W7tnJdL...,Qonli tatil\n\nTili: oʻzbek tilida\nJanri: kom...,1009,2,Qonli tatil,2221.jpg
2,2222,BAACAgEAAx0Ec8rVjQACCK5kZJmLSq-hTSSIxuVZyqhU--...,Qo'rqmas\n\nTili: O'zbek tilida\nSifati 480p\...,671,3,Qorqmas,2222.jpg
3,2223,BAACAgIAAx0Ec8rVjQACCK9kZJmM3EGy87VlBnsuiyEk4L...,#tarjima #romantik #melodramma\n\nNomi: Shamol...,539,4,tarjima romantik melodramma,2223.jpg
4,2224,BAACAgQAAx0Ec8rVjQACCLBkZJmNqK19bYLVLzBEIpKq-v...,Nomi: Qonli O'yin yohud qimirlagan O'ladi\n\nD...,370,5,Qonli Oyin yohud qimirlagan Oladi,2224.jpg
...,...,...,...,...,...,...,...
286,2506,BAACAgIAAx0Ec8rVjQACCcpkZJ5Yf-nkMzrWpF3Z3laa34...,Mister Alfons o'zbek film\n\n,279,287,Mister Alfons,2506.jpg
287,2507,BAACAgIAAx0Ec8rVjQACCctkZJ5YSyz9Yq7Y7qE3t_C_lg...,Menga esa sen kerak \n\n,408,288,Menga esa sen kerak,2507.jpg
288,2508,BAACAgQAAx0Ec8rVjQACCcxkZJ5ZH1JeeR1qh4NDi-U3cL...,Shallaqi (o'zbek film) \n\n,378,289,Shallaqi,2508.jpg
289,2509,BAACAgQAAx0Ec8rVjQACCc1kZJ5ZgFmvGKi7oReLx--9cq...,3-kvartal (o'zbek film) \n\n,240,290,3-kvartal,2509.jpg


# 6. Save the data

In [158]:
dataset.to_csv('dataset/movies.csv', index = False)